In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('https://github.com/feifeipan/Kaggle-Forest-Cover-Type-Prediction/raw/master/input/train.csv')
test_df = pd.read_csv('https://github.com/feifeipan/Kaggle-Forest-Cover-Type-Prediction/raw/master/input/test.csv')
# train_df = pd.read_csv('Data/train.csv')
# test_df = pd.read_csv('Data/test.csv')

In [3]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [5]:
X_train = train_df.drop("Cover_Type", axis=1)
Y_train = train_df["Cover_Type"]
X_test  = test_df.drop(['Id'], axis=1).copy()

In [6]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [7]:
parameters = {'n_estimators': [5, 50, 100],
              'learning_rate': np.linspace(0.02,0.16,4),
              'num_leaves': [31, 61],
              'min_data_in_leaf': [20, 30, 40],
              'max_depth': range(3,8),
}

lgbm_model = GridSearchCV(lgbm.LGBMClassifier(), parameters, cv=5).fit(X_train, Y_train)

print(accuracy_score(Y_train, lgbm_model.predict(X_train)))
print(lgbm_model.best_score_)
print(lgbm_model.best_params_)
print(lgbm_model.best_estimator_)

Y_pred = lgbm_model.predict(X_test)
acc_log = round(lgbm_model.score(X_train, Y_train) * 100, 2)
print('ended', acc_log)

0.7294973544973545
0.5898148148148149
{'learning_rate': 0.16, 'max_depth': 4, 'min_data_in_leaf': 20, 'n_estimators': 5, 'num_leaves': 31}
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.16, max_depth=4,
               min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=20, min_split_gain=0.0, n_estimators=5,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


ValueError: Number of features of the model must match the input. Model n_features_ is 55 and input n_features is 54 

In [8]:
Y_pred = lgbm_model.predict(test_df)
acc_log = round(lgbm_model.score(X_train, Y_train) * 100, 2)
print('ended', acc_log)

ended 72.95


In [9]:
submission = pd.DataFrame({
        "Id": test_df["Id"],
        "Cover_Type": Y_pred
    })
submission.to_csv('forest_solution_LightGBM-1.csv', index=False)

In [10]:
submission[['Id', 'Cover_Type']].groupby(['Cover_Type'], as_index=False).count().sort_values(by='Cover_Type', ascending=False)

,Cover_Type,Id
6,7,50626
5,6,33853
4,5,119269
3,4,7105
2,3,28305
1,2,73421
0,1,253313
